# Insert new data into texas.weather_county

New data was found! Insert it into the table

In [ ]:
import psycopg2
import pandas as pd
from aws_secrets import get_secret

In [ ]:
def create_rw_conn(secrets):
    # pass along secrets to pyscopg2
    ENDPOINT = secrets['ENDPOINT']
    PORT = secrets['PORT']
    USER = secrets['USER']
    PASSWORD = secrets['PASSWORD']
    DATABASE = secrets['DATABASE']
    
    # create connection string    
    conn = psycopg2.connect(host=ENDPOINT, port=PORT, user=USER, 
        database=DATABASE, password=PASSWORD, sslmode='prefer', 
        sslrootcert="[full path]rds-combined-ca-bundle.pem")
    return conn


def fetch_results(sql, conn):
    cur = conn.cursor()
    cur.execute(sql)
    columns = [desc[0] for desc in cur.description]
    df = pd.DataFrame(cur.fetchall(), columns=columns)
    cur.close()
    return df


In [ ]:
# connect using a read/write user
secrets = get_secret()
conn = create_rw_conn(secrets=secrets)

In [ ]:
sql = """
    INSERT INTO texas.weather_county(fips_code, county_name, temperature)
    VALUES ('48129', 'Donley', 69);
    """

In [ ]:
cur = conn.cursor()
cur.execute(sql)
cur.close()

In [ ]:
conn.close()
conn = create_rw_conn(secrets=secrets)

In [ ]:
sql = "SELECT * FROM texas.weather_county;"
df = fetch_results(sql=sql, conn=conn)
df.head(10)

## UPSERT: INSERT/UPDATE

Insert new records, and update existing records

In [ ]:
sql = """
    INSERT INTO texas.weather_county(fips_code, county_name, temperature)
    VALUES ('48113', 'Dallas', 66), ('48225', 'Houston', 66), ('48023', 'Baylor', 66), ('48251', 'Johnson', 66)
    ON CONFLICT (fips_code) DO UPDATE
    SET
        temperature=excluded.temperature,
        updated_at=NOW()
    ;
    """

In [ ]:
cur = conn.cursor()
cur.execute(sql)

conn.commit()

cur.close()


In [ ]:
sql = "SELECT * FROM texas.weather_county;"
df = fetch_results(sql=sql, conn=conn)
df.head(10)

In [ ]:
conn.close()